In [1]:
import pandas as pd
import numpy as np
import glob
import time
from sqlalchemy import create_engine
import psycopg2
import pyodbc
import yaml
import os

In [2]:
# setting up a database connection

driver = 'SQL Server'
server = 'localhost'
database = 'cars'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};Server='+server+';Database='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

engine = create_engine('mssql+pyodbc://@' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server', fast_executemany=True)

# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
# driver = 'ODBC Driver 17 for SQL Server' 

# username = 'myusername' 
# password = 'mypassword' 
# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
# cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)

In [3]:
def create_database_connection():

    """
    Establish connection to the database 
    Return the connection and cursor refrence
    :return: returns (cur, conn) a cursor and connection reference
    """

    # Opening the YAML config file
    with open("mssql_config.yml") as f:
        content = f.read()

    # from config.yml import user name and password
    config = yaml.load(content, Loader=yaml.FullLoader)
    
    cnxn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes', fast_executemany=True, pool_pre_ping=True
        )
    cur = cnxn.cursor()
    
    return cur, cnxn


def drop_tables(cur, cnxn):

    # cur.execute("""CREATE SCHEMA cars """)

    cur.execute(""" DROP TABLE IF EXISTS cars.bodytype""")
    cur.execute(""" DROP TABLE IF EXISTS cars.categories""")
    cur.execute(""" DROP TABLE IF EXISTS cars.condition""")
    cur.execute(""" DROP TABLE IF EXISTS cars.listing""")
    cur.execute(""" DROP TABLE IF EXISTS cars.trueprices""")
    cnxn.commit()
    print("Successfully dropped tables")

In [4]:
filepath = r"Datasets"

def process_data(filepath):

    """
    import the datasets from where they are (locally or on the web) directly to the database
    - In this case, the datasets are sitting locally in CSV files on my machine
    """


    # get all files matching extension from directory
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root + '/*.csv'))

        for file in files:
            df = pd.read_csv(file, header = 0, delimiter=";", decimal = ",", index_col=False)

            df.to_sql(f'{file[9:-4]}', engine, schema='cars', if_exists='append', index=False, chunksize=1000000)
            print(f"data transformed and inserted for {file}")

In [5]:
def database_main():

    cur, cnxn = create_database_connection()
    drop_tables(cur, cnxn)
    cnxn.close()

In [6]:
if __name__ == "__main__":

    database_main()
    process_data(filepath)

Successfully dropped tables
data transformed and inserted for Datasets\bodytype.csv
data transformed and inserted for Datasets\categories.csv
data transformed and inserted for Datasets\condition.csv
data transformed and inserted for Datasets\listing.csv
data transformed and inserted for Datasets\trueprices.csv
